# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# from plotly.subplots import make_subplots
import requests
import json

In [2]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['am_ig_user_id']
access_token = credentials['am_ig_access_token']

# `get_ig_account_insights`

In [3]:
from datetime import time, datetime, timedelta
import re
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
    since='2022-01-01', until='2023-03-01', filename='am_IG_account_insights_2022-01-01_2023-03-01')

https://graph.facebook.com/v15.0/17841401256046961/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675065600.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Requesting page 3...
	Response status code:  200
Requesting page 4...
	Response status code:  200
Requesting page 5...
	Response status code:  200
Requesting page 6...
	Response status code:  200
Requesting page 7...
	Response status code:  200
Requesting page 8...
	Response status code:  200
Requesting page 9...
	Response status code:  200
Requesting page 10...
	Response status code:  200
Requesting page 11...
	Response status code:  200
Requesting page 12...
	Response status code:  200
Requesting page 13...
	Response status code:  200
Requesting page 14...
	Response status code:  200
Requesting page 15...
	Response status code:  200
Number of days of data: 451
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/onli

In [4]:
df

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,11968,2023-01-30T08:00:00+0000,5694,2023-01-30T08:00:00+0000
1,13255,2023-01-31T08:00:00+0000,5340,2023-01-31T08:00:00+0000
2,12382,2023-02-01T08:00:00+0000,4913,2023-02-01T08:00:00+0000
3,11649,2023-02-02T08:00:00+0000,5221,2023-02-02T08:00:00+0000
4,13558,2023-02-03T08:00:00+0000,5155,2023-02-03T08:00:00+0000
...,...,...,...,...
446,13059,2021-12-31T08:00:00+0000,1974,2021-12-31T08:00:00+0000
447,5812,2022-01-01T08:00:00+0000,2313,2022-01-01T08:00:00+0000
448,15904,2022-01-02T08:00:00+0000,2411,2022-01-02T08:00:00+0000
449,5100,2022-01-03T08:00:00+0000,2155,2022-01-03T08:00:00+0000


# process_account_insights

In [5]:
def process_df_timestamp(input_df, timestamp_column='timestamp'):
    """
    Convert dates in the json-derived dataframe from Facebook API read requests
    into different formats.

    Parameters: 
        - input_df : DataFrame with the timestamp of the data.
        - timestamp_column (str): Name of the column with the timestamp.
    """
    df = input_df.reset_index(drop=True)
    regex_date = r'.+T'
    df[timestamp_column] = pd.to_datetime(df[timestamp_column])
    df['date'] = df[timestamp_column].dt.date
    df['year'] = df[timestamp_column].dt.year
    df['month'] = df[timestamp_column].dt.month
    df['week_of_year'] = df[timestamp_column].dt.isocalendar().week
    df['day_of_week'] = df[timestamp_column].dt.dayofweek
    df['day_of_week_name'] = df[timestamp_column].dt.day_name()
    df['time'] = df[timestamp_column].dt.time
    df['hour'] = df[timestamp_column].dt.hour
     ## SH 2023-02-27 18:00 new
    df['year-month'] = df[timestamp_column].dt.to_period('M').dt.start_time # first day of the month
    df['year-week'] = df[timestamp_column].dt.to_period('W').dt.start_time # First day of the week (same day of week as for Jan 1 of that year)

    return df

In [21]:

def process_account_insights(
    input_df, metric_column_suffix='value', timestamp_column_suffix='end_time',
    timezone='Canada/Pacific'
    ):

    timestamp_columns = input_df.filter(regex=timestamp_column_suffix).columns
    # Check to see if all the timestamp columns are identical before proceeding
    for index, column in enumerate(timestamp_columns[:-1]): 
        if input_df[column].equals(input_df[timestamp_columns[index+1]]):
            continue
        else:
            print('Error: timestamp columns are not all identical')
            break
    df = pd.concat([
        input_df[column], 
        input_df.filter(regex=metric_column_suffix)
        ], axis=1)
    if timezone:
        print('Time zone:', timezone)
        df[column] = pd.to_datetime(df[column]).dt.tz_convert(timezone)
    else:
        print('Time stamps in UTC time')
    timezone_suffix = timezone if timezone else 'UTC'
    timestamp_column_name = f'metric_{timestamp_column_suffix}_{timezone_suffix}'
    df = df.rename(columns={column: timestamp_column_name})
    df = process_df_timestamp(df, timestamp_column=timestamp_column_name) 
    
    return df.sort_values(timestamp_column_name).reset_index(drop=True)


In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def plot_account_insights(
        input_df, agg='sum',
        metric_column_suffix='value', timezone='Canada/Pacific',
        streamlit=False):
    df = process_account_insights(input_df, timezone=timezone)
    metrics = df.columns[df.columns.str.contains('_'+metric_column_suffix)].tolist()
    metrics = [metric.replace('_'+metric_column_suffix, '') for metric in metrics]
    df.columns = df.columns.str.replace('_'+metric_column_suffix, '')
    # groupby_options = ['year-month', 'year-week', 'date', 'day_of_week']
    groupby_options = ['year-month', 'year-week', 'day_of_week', 'date']
    subplot_titles = [
        f'{metric} per {groupby} ({agg})' for groupby in groupby_options for metric in metrics]
    fig = make_subplots(
        rows=len(groupby_options)*len(metrics), cols=1, subplot_titles=subplot_titles
        )
    row = 1
    xtick_list = []
    df_list = []
    for groupby in (groupby_options):
        df_grouped = df.filter(items=metrics+[groupby]).groupby(
            groupby).agg(agg) if groupby !='date' else df.set_index('date')
        if groupby == 'day_of_week':
            day_names = df.sort_values('day_of_week')['day_of_week_name'].unique()
            df_grouped.index = [day[:3] for day in day_names]
        df_list.append(df_grouped)
        for metric in (metrics):
            fig.add_trace(
                go.Scatter(
                    y=df_grouped[metric],
                    x=df_grouped.index, showlegend=False,
                    
                ),
                row=row, col=1
            )
            row += 1
            if len(df_grouped.index) <= 24:
                xtick_list.append(df_grouped.index)
            else:
                xtick_list.append(None)

    fig.update_layout(
        title_text=f'Insights (periods end at {df.loc[0,"hour"]}:00 {timezone} time)',
        title_xanchor='center', title_x=0.5,
        height = len(groupby_options)*len(metrics) * 200,
        # margin_b=120, margin_l=50, margin_r=50,
        )
    # Update the xtick labels for each subplot
    for ax in fig['layout']:
        if ax.startswith('xaxis'):
            subplot = int(ax[5:]) if ax[5:] else 1
            fig['layout'][ax]['tickvals'] = xtick_list[subplot-1]
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return fig

plot_account_insights(df, timezone='Australia/Sydney', agg='mean');

Time zone: Australia/Sydney


C:\Users\silvh\AppData\Local\Temp\ipykernel_24936\979048190.py:22: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

C:\Users\silvh\AppData\Local\Temp\ipykernel_24936\979048190.py:23: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



# *End of Page*